# Optimization of linear signal processing in photon counting lidar
This notebook was developed as a suppliment to the publication
Optimization of photon counting lidar signal processing through Poisson thinning
by Matthew Hayman
submitted to Optics Letters *In Review*, DOI:TBD

The notebook demonstrates the basic concepts for optimizing linear smoothing
kernenls to a particular lidar scene by splitting an observed photon count
profile into a "fit" and "verification" profile.  Smoothing kernels are applied
to the "fit" profile then evaluated against the verification data set to find 
the optimal filter kernel.

The code and functions contained in this notebook are available for public use
so long as the original publication is referenced in any published work

The data used in this example is from an NCAR MicroPulse DIAL (MPD) [1] using the potassium HSRL channels [2].  This 
data is provided for example purposes only and should not be used in scientific
study.  No data quality assurance can be provided for this dataset.

1. Spuler et al., "Field-deployable diode-laser-based differential absorption lidar (DIAL) for profiling water vapor," Atmos. Meas. Tech., 8, 1073–1087, DOI:10.5194/amt-8-1073-2015 (2015).

2. Stillewell et al.,"Demonstration of a combined differential absorption and high spectral resolution lidar for profiling atmospheric temperature," Opt. Express, 28, 71-93, DOI: 10.1364/OE.379804 (2020).

In [ ]:
# import python libraries
import os
import sys
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.colors import LogNorm

In [ ]:
# TODO: set relative path
# os.path.abspath('../../')
data_path = '../data/'

In [ ]:
ncfile = 'mpd05.20181022T12300019921_20181022T15163019921.nc'
# ncfile = 'mpd05.20181022T18032019921_20181022T20495019531.nc'

In [ ]:
# list of lidar profile names to be loaded
lidar_profile_data = ['Combined_Counts','Molecular_Counts']

In [ ]:
"""
Function definitions for optimizing the filter
"""

def poisson_thin(pc_data):
    """
    Poisson thin photon counting data
    
    inputs:
        pc_data - array of raw photon count data
            This is assumed to have a Poisson PDF
    outputs:
        pc_1,pc_2 - the resulting thinned photon count
            arrays
    """
    # the first thinned profile is calculated using a binomial random
    # number generator to simulate 
    # flipping a coin to decide if a photon count is included in this
    # profile or not
    pc_1 = np.random.binomial(pc_data.astype(np.int),0.5,size=pc_data.shape)
    # the second profile is whatever photon counts are left
    pc_2 = pc_data-pc_1
    
    return pc_1,pc_2

def build_Gaussian_kernel(sigt,sigz,norm=True):
    """
    Generates a Gaussian convolution kernel for
    standard deviations sigt and sigz in units of grid points.
    sigt and sigz are defined in units of grid steps
    """        

    
    nt = np.round(4*sigt) # estimate size of time grid needed
    nz = np.round(4*sigz) # estimate size of range grid needed
    t = np.arange(-nt,nt+1) # create time grid     
    z = np.arange(-nz,nz+1) # create range grid
    
    
    # build Gaussian kernel in time
    kconv_t = np.exp(-t**2*1.0/(sigt**2))
    
    # check for singularities in the definition
    # if they exist, make the filter a delta function in time
    if kconv_t.size > 1:
        if np.sum(kconv_t) == 0:
            it0 = np.argmin(np.abs(t))
            kconv_t[it0] = 1.0
    else: 
        kconv_t = np.ones(1)

    # build Gaussian kernel in range
    kconv_z = np.exp(-z**2*1.0/(sigz**2))
    
    # check for singularities in the definition
    # if they exist, make the filter a delta function in range
    if kconv_z.size > 1:
        if np.sum(kconv_z) == 0:
            iz0 = np.argmin(np.abs(z))
            kconv_z[iz0] = 1.0
    else:
        kconv_z = np.ones(1)

    # combine the time and range kernels
    kconv = kconv_t[:,np.newaxis]*kconv_z[np.newaxis,:]

    # normalize the area of the kernel to conserve energy
    if norm:
        kconv = kconv/(1.0*np.sum(kconv))

    return kconv

In [ ]:
"""
Load the photon counts from the netcdf file
"""


profs = {}
with xr.open_dataset(data_path+ncfile) as ds:
    for lvar in lidar_profile_data:
        profs[lvar] = ds[lvar].values
    lidar_range = ds['range'].values.copy()
    time = ds['time'].values.copy()
    

In [ ]:
"""
This notebook will run a demonstration case on the combined
channel of the MPD HSRL operating at 770.  This is effectively
equivilant to a backscatter lidar observation.
"""
demo_var = 'Combined_Counts'

There are a variety of features in the scene from this data file.  Thick low clouds are between time indices 1000 and 2000.  High ice clouds are seen intermittantly starting around time index 2500 until lower clouds arrive late in the day near time index 7500.  The sun rises around time index 4800 and several instances of high background occur due to the simultaneous presence of sun and clouds.

In [ ]:
# plot a quicklook of the backscatter data
fig,ax = plt.subplots(1,1,figsize=(18,4))
ax.imshow(profs[demo_var][:,::-1].T,norm=LogNorm())

The demonstration presented here focuses on optimization of range smoothing kernels, but kernels can similarly be optimized for time smoothing or 2D kernels consisting of range and time components.  

Generally we perform both time and range optimization independently where each time bin has its own unique smoothing kernel and each range bin has its own uniquely determined time smoothing kernel.

In order to compare methodology, thinning is performed once and different methods are subsequently applied to that same data.  Comparing results between two different thinned cases, even if they originate from the same profile is generally not effective because the offset in the inverse log-likelihood is prone to variation between different thinning operations.

In [ ]:
itime =  400 # select a profile index for this demonstration case 
              # interesting cases: 
              #   night light clouds: 3500
              #   daytime/high background medium clouds: 5500
              #   dattime/high background dense clouds: 7500
              #   daytime/high background, low alt aerosols, thin cirrus:  7000

max_alt = 12e3

imax_alt = np.argmin(np.abs(lidar_range-max_alt))
            
# assign the demonstration case to its own independent variable
pdemo = (profs[demo_var][itime,3:imax_alt])[np.newaxis,:]  # remove bottom three bins from laser "bang"
plidar_range = lidar_range[3:imax_alt]  # create a new range array for the bang-removed profile
dz = np.mean(np.diff(plidar_range))  # store the range resolution

# Poisson Thinning
Poisson thinning generates two statistically independent observations from one observation.  The resulting thinned profiles have the same underlying mean photon arrival rate driving the observations.  This is demonstrated below on the two thinned profiles are
```
pfit
```
used for applying the filter ($\mathbf{f}$ in the publication) and 
```
pver
```
used for scoring the the filtered result ($\mathbf{g}$ in the publication).

If the two observations have equal signals but uncorrelated noise, then
$$E\left[\frac{\mathbf{f}-\mathbf{g}}{\sqrt{\sigma_f^2+\sigma_g^2}}\right] \approx 0 $$
and 
$$std\left[\frac{\mathbf{f}-\mathbf{g}}{\sqrt{\sigma_f^2+\sigma_g^2}}\right] \approx 1 $$

where for a Poisson observation $f$, the variance in the estimate of mean photons is given by $\sigma_f^2 = f+1$

In [ ]:
# Poisson thin the raw photon counts of the profile
pfit,pver = poisson_thin(pdemo)

In [ ]:
# plot the thinned data
# the two verify and fit data should have 
# statistically independent noise but common 
# underlying signals
fig,ax = plt.subplots(1,1,figsize=(5,5))
ax.plot(pdemo.flatten(),plidar_range,'k.',label='observation')
ax.plot(pfit.flatten(),plidar_range,'.',alpha=0.5,label='fit data')
ax.plot(pver.flatten(),plidar_range,'.',alpha=0.5,label='verify data')
ax.set_xscale('log')
ax.set_ylabel('Range [m]')
ax.set_xlabel('Photon Counts')
ax.grid(b=True)
ax.set_ylim([0,12e3])
ax.legend()

The original photon count data, thinned fit data, and thinned verification data are shown above.  The shapes of the thinned profiles are the same but the statistical noise is not.

In [ ]:
# Plot the difference in photon counts between the
# fit and verify profiles 
# compare this to the expected standard deviation
# for uncorrelated Poisson observations
fig,ax = plt.subplots(1,1,figsize=(5,5))
ax.plot(pfit.flatten()-pver.flatten(),plidar_range,'g.',label='Actual')
ax.plot(np.sqrt(pfit+1+pver+1).flatten(),plidar_range,'k--',label='Expected std.')
ax.plot(-np.sqrt(pfit+1+pver+1).flatten(),plidar_range,'k--')
ax.set_ylabel('Range [m]')
ax.set_xlabel('Photon Count Difference')
ax.grid(b=True)
ax.set_ylim([0,12e3])
ax.legend()

# adjusting by the expected uncertainty results in a
# a mean value near zero with
# a standard deviation near one
# further suggesting the signals are common but the
# noise is statistically independent
print('mean: %f'%np.mean((pfit-pver)/np.sqrt(pfit+1+pver+1)))
print('std: %f'%np.std((pfit-pver)/np.sqrt(pfit+1+pver+1)))

The above plot shows the difference between the two thinned profiles (green dots) with the theoretically estimated variance in the difference assuming both profiles are observations of Poisson random variables and they are uncorrelated. ($\sqrt{\sigma_f^2+\sigma_g^2}$).  The theoretically estimated standard deviation appears representative of the scatter in the difference signal and the scattered signal appears to be zero mean.

The standard deviation normalized mean and standard deviation are reported above these figures.  The mean is near zero and the standard deviation is near one.

# Find the Optimal Filter
The optimal filter kernel in a set is found by applying all the kernels to $\mathbf{f}$ then scoring the result $\tilde{\mathbf{f}}$ against the verification profile $\mathbf{g}$.  The score is calculated using the inverse log-likelihood
$$\mathcal{E} = \tilde{\mathbf{f}} - \mathbf{g}\ln \tilde{\mathbf{f}} $$
The filter that produces the lowest inverse log-likelihood is taken to be the best filter from the set for this particular scene.

In [ ]:
"""
Filter the raw profile
"""

# define range of filters to try
filt_size = np.logspace(-1,1,40)

# initialize the output arrays
inv_ll = np.zeros(filt_size.size)
filt_profs = np.zeros((filt_size.size,pfit.size))

for ifilt,filter_width in enumerate(filt_size):
    # get the convolution kernel
    kern = build_Gaussian_kernel(0,filter_width)
    
    # normalize by the amount of points in a region
    # to avoid edge effects
    norm = np.ones(pfit.shape)
    norm = scipy.signal.convolve2d(norm,kern,mode='same')
    # apply the filter and normalize the result
    # with the amount of data points contributing
    pfilt = scipy.signal.convolve2d(pfit,kern,mode='same')/norm
    
    # save the results
    filt_profs[ifilt,:] = pfilt
    inv_ll[ifilt] = np.nansum(pfilt-pver*np.log(pfilt))

# get the index to the solution
isol = np.argmin(inv_ll)

In [ ]:
# plot the resulting optimized profile over the raw data
# also include an over-filtered example
# in a separate plot show the inverse log-likelihood as a function of filter width

fig,ax = plt.subplots(2,1,figsize=(5,10))
ax[0].plot(pver.flatten(),plidar_range,'.',label='raw data')
ax[0].plot(filt_profs[isol,:],plidar_range,'k--',label='filtered')
ax[0].plot(filt_profs[-1,:],plidar_range,':',label='over filtered')
ax[0].set_xscale('log')
ax[0].set_ylabel('Range [m]')
ax[0].set_xlabel('Photon Counts')
ax[0].grid(b=True)
ax[0].set_ylim([0,12e3])
ax[0].legend()

ax[1].plot(filt_size*dz,inv_ll,'b.')
ax[1].plot(filt_size[isol]*dz,inv_ll[isol],'gs')
ax[1].set_xscale('log')
ax[1].grid(b=True)
ax[1].set_xlabel('Filter Width [m]')
ax[1].set_ylabel('Inverse Log-Likelihood')

#ax[1].set_ylim([inv_ll.min()*1.1,0.01*np.median(inv_ll)])

The top figure shows the verification data in blue dots and the optimally filtered data is in the blacked dashed line.  For comparison, an over filtered case is also shown as the orange dotted line.  Overfiltering is more effective as supressing the random errors, but also skews the signal, particularly where clouds are present.

The bottom figure shows the inverse log-likelihood for each filter kernel width (defined by the standard deviation of a Gaussian) where the minimum (optimal) value is indicated by the green square.

# 2D Optimization for Backscatter Ratio
Principles described above can be similarly applied to multiple channels in two dimensions.

In [ ]:

max_alt = 12e3

imax_alt = np.argmin(np.abs(lidar_range-max_alt))

hsrl_profs = {}
for var in lidar_profile_data:
    # assign the demonstration case to its own independent variable
    ptemp = profs[var]      # remove bottom three bins from laser "bang"
    hsrl_profs[var] = {}
    hsrl_profs[var]['raw'] = ptemp.copy()
    hsrl_profs[var]['bg'] = 0.5*np.mean(hsrl_profs[var]['raw'][:,-100:],axis=1)[:,np.newaxis]
    
    hsrl_profs[var]['raw']=hsrl_profs[var]['raw'][:,3:imax_alt]
    
    # Poisson thin the raw photon counts of the profile
    hsrl_profs[var]['fit'],hsrl_profs[var]['ver'] = poisson_thin(hsrl_profs[var]['raw'])
    
hsrl_lidar_range = lidar_range[3:imax_alt]  # create a new range array for the bang-removed profile
dz = np.mean(np.diff(hsrl_lidar_range))  # store the range resolution  
dt = np.mean(np.diff(time))  # store the time resolution  

In [ ]:
backscatter_ratio = (hsrl_profs['Combined_Counts']['fit']-hsrl_profs['Combined_Counts']['bg'])/ \
                        (hsrl_profs['Molecular_Counts']['fit']-hsrl_profs['Molecular_Counts']['bg'])

In [ ]:
plt.figure()
plt.imshow(backscatter_ratio[:,::-1].T,norm=LogNorm())
plt.clim([1,1e3])